In [23]:
# imports:
from  random import randint ,choice ,random ,choices
import numpy as np
import copy 
from math import floor
# from sklearn.metrics import mean_squared_error
# from math import inf
# import pysnooper

In [24]:
# global variables:
unary_operators_list =list(["sin" , "cos" , "tan" , "cot" , "exp" ])
binary_operators_list =list(["+" , "-" , "/" , "*" , "**"])

# user's values:
operators_list =["+" , "-" ]*5+\
["/" , "*" ]* 4 +\
[ "**"  ]*0 +\
[ "sin" ,"cos" , "tan" , "cot" , "exp"]*0   #user can change the quantity of operators based on function determination of complexity.
users_variables = ["x"]             #what variables our function does have
min_operatos_num = 1                #determine how many operators do we need in our function(create n operators between these 2 variables.)
max_operatos_num =5
variable_or_constant_factor = 0.3   #a factor that determines to use constant or a variale in leaf node of the tree .the closer value to 0 the more constants will appear in your function.(must be between 0 and1)
depth_or_breadth_factor = 0.5       #a factor that determines to fill both sides of the operator on not. the more closer to 0 tree will get filled in breadth not depth. (must be between 0 and1)
                                    # for example: closer to 0: (((x*10)+2)-5)/3      closer to 1: ((x*10)+2) - ((x-5)\3)
min_constant_limit = 0              #determine that constants must be in what period.
max_constant_limit =50            
num_of_population_in_a_gen = 50     #how many trees should be included in each generation.
num_of_gens =3                      #how many geneerations should be excluded
# function values:
sample_values = {                   #user must enter some values in spots to 
    "x" : [i for i in range(1 , 10)] ,
    # "outputs" : [0,1,4,9,16]        #can be skipped with replacing function in function below.
}
real_function_str = "x**2 + 1 "      #what function is.

In [25]:
# funtion to  fill sample["output"] for 1 variable:
def fill_sample_output(func_str = real_function_str):
    temp_list = []
    for x_val in sample_values["x"]:
        temp_str = func_str.replace("x" , str(x_val))
        temp_val =eval(temp_str)
        temp_list.append(temp_val)
    sample_values["outputs"]  =temp_list
    # return temp_list

In [26]:
fill_sample_output()

In [27]:
class Node():
    def __init__(self , value=None , used=0, father=None , right=None ,left=None , depth = 0 ):
        self.father =father
        self.right = right
        self.left = left
        self.depth =depth
        self.used = used
        self._value = value
        # self.is_operator = new_value in operators_list
        self.is_op()
        self.is_binary_op()
    
    def set_random_op(self):
        new_value= choice(operators_list)                
        self.value=new_value 
    def is_binary_op(self):
        self.is_binary = self.value in binary_operators_list
    def is_op(self  ):
        self.is_operator = self.value in operators_list
    
    @property
    def value(self):
        return self._value 
    @value.setter
    def value(self, new_value=None):
        self._value = new_value
        self.is_operator = new_value in operators_list
        self.is_binary = new_value in binary_operators_list
        # self.is_binary()
        # self.is_op()
        
    def __repr__(self) -> str:
        return f"(V:{self.value} L:{self.left} R:{self.right})"
    def __str__(self) -> str:
        if self.left == None and self.right==None :                                          #constant or variable
            return str(self.value)
        elif self.value in binary_operators_list:                                            #binary operator
            return  f"({self.left} {self.value} {self.right})"
        else:                                                                                #unary operator                                        
            return f"{self.value} ({self.left}) "

In [28]:
class Tree():
    def __init__(self ,  root:Node , nodes:list , MSE = -100000 ):
        self.root = root 
        self.MSE = MSE
        self.nodes = list(nodes)
        # self.nodes.append(root)
        
    def remove_node(self , node:Node):  #note that should handle node's father outside of this function 
        if node.left==None:
            self.nodes.remove(node)
            pass
        else:
            self.nodes.remove(node)
            self.remove_node(node.left)
            self.remove_node(node.right) 
    
    def add_node(self , node:Node): #note that should handle node's father outside of this function
        if node.left==None:
            self.nodes.append(node)
            pass
        else:
            self.nodes.append(node)
            self.add_node(node.left)
            self.add_node(node.right)
        
    def replace_node(self ,node_to_replace:Node , node_that_will_be:Node):  
        father_node = node_to_replace.father
        target_node = copy.deepcopy(node_that_will_be)
        target_node.father = father_node
        if not father_node=="is_root": 
            if father_node.right==node_to_replace :
                father_node.right = target_node
            else:
                father_node.left = target_node
        for node in self.nodes:
            if node == node_to_replace:
                self.nodes.remove(node)
        self.add_node(target_node)
        
    def mutate(self):
        print("Mutated!!!")
        node = choice(self.nodes)
        if node.left==None:
            if random()>0.2:
                node.value = randint(min_constant_limit ,max_constant_limit)
            else:
                node.value = choice(users_variables)
        else:
            node.value = choice(operators_list)
    
    def evaluate(self ,x_val ,node:Node = None ):
        if node == None:
            node = self.root
        # if node.is_operator and node.is_binary :                                                #**note that it doesnt work on unary operators yet!
        #     if node.value =="+":    
        #         return np.add(self.evaluate(node.left) , self.evaluate(node.right))
        #     elif node.value == "-" :
        #         return np.subtract(self.evaluate(node.left) , self.evaluate(node.right))
        #     elif node.value =="/":
        #         return np.multiply(self.evaluate(node.left) , self.evaluate(node.right))
        #     elif node.value == "*":
        #         return np.multiply(self.evaluate(node.left) , self.evaluate(node.right))
        #     elif node.value == "**":
        #         return np.multiply(self.evaluate(node.left) , self.evaluate(node.right))
        # elif not node.is_operator and not node.is_binary:
        #     if node.value not in users_variables:
        #         return np.single(node.value)
        #     else:
        #         return np.single(x_val)
        if node.is_operator and node.is_binary :                                                #**note that it doesnt work on unary operators yet!
            if node.value =="+":    
                return (self.evaluate(node.left) + self.evaluate(node.right))
            elif node.value == "-" :
                return (self.evaluate(node.left) - self.evaluate(node.right))
            elif node.value =="/":
                return (self.evaluate(node.left) / self.evaluate(node.right))
            elif node.value == "*":
                return (self.evaluate(node.left) * self.evaluate(node.right))
            elif node.value == "**":
                return (self.evaluate(node.left) ** self.evaluate(node.right))
        else:
            if node.value not in users_variables:
                # return np.single(node.value)
                return float(node.value)
            else:
                return float(x_val)
            
                
    def __repr__(self) -> str:
        return f"tree[ {self.root} ] "
    def __str__(self) -> str:
        return f"{self.root}"

In [29]:
def create_random_tree():
    num_of_ops = randint(min_operatos_num , max_operatos_num)
    unattached_nodes =list([Node() for _ in range(num_of_ops-1)])
    root_node = Node(father="is_root",used=0 ,depth=0,value=choice(operators_list))          
    attached_nodes= list([root_node])                                                                       #a node gets attached when it has a parent
    # creating a random tree with operators:
    # print("test r:",root_node ,"n.o.o:",num_of_ops , "a:", attached_nodes , "u:" , unattached_nodes )     #>>>for logging
    while unattached_nodes !=[]:
        # print("***first loop")                                                                            #>>>for logging
        node =choice(attached_nodes)
        if node.used == 0:                                                                                  #node is binary operator.
            if node.value == None:                                                                          #sometimes it is already set in previous loop.
                node.value = choice(operators_list)            
            if node.value in binary_operators_list:
                if random()< depth_or_breadth_factor and len(unattached_nodes)>1:                           #fill both childs
                    child1 = unattached_nodes.pop()
                    child2 = unattached_nodes.pop()
                    child1.set_random_op()
                    child2.set_random_op()
                    node.left =child1
                    node.right =child2
                    child2.depth , child1.depth =node.depth +1,node.depth +1
                    child1.father , child2.father = node  , node
                    node.used = 1
                    attached_nodes.append(child1)
                    attached_nodes.append(child2)
                else:                                                                                       #fill one child
                    child = unattached_nodes.pop()
                    child.set_random_op()
                    if random()<0.5:
                        node.left = child
                    else:
                        node.right = child
                    child.depth =node.depth +1
                    node.used =1
                    child.father = node
                    attached_nodes.append(child)
            else:                                                                                          #node is unary operator.
                child = unattached_nodes.pop()
                child.set_random_op()
                node.left =child
                child.depth =node.depth +1
                node.used=1
                child.father = node
                attached_nodes.append(child)
    # print("after op nodes set: a:" ,attached_nodes ,"u:",unattached_nodes )                              #>>>for logging
    new_attached_nodes = []
    # adding values and constants to leaf nodes
    for node in attached_nodes:
        # print("in second loop; a:" ,attached_nodes ,"u:",unattached_nodes )                              #>>>for logging
        if node.right ==None :                                                                             #right node is empty  
            choose_const_or_var = random()                                                                 # **note:think that it woulld get into troubles when it comes to unary operators.
            if  choose_const_or_var>variable_or_constant_factor:                                           #fill with constant
                temp_value = randint(min_constant_limit , max_constant_limit)                                                           
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)                        
                node.right = temp_node
                new_attached_nodes.append(temp_node)
            else:                                                                                          #fill with variable                                  
                temp_value = choice(users_variables)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.right = temp_node
                new_attached_nodes.append(temp_node)
        if node.left ==None :                                                                              #left node is empty
            choose_const_or_var = random()
            if choose_const_or_var>variable_or_constant_factor:                                            #fill with constant
                temp_value = randint(min_constant_limit , max_constant_limit)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.left = temp_node
                new_attached_nodes.append(temp_node)
            else:                                                                                          #fill with variable                                  
                temp_value = choice(users_variables)
                temp_node = Node(value=temp_value ,father=node , depth=node.depth+1)
                node.left = temp_node
                new_attached_nodes.append(temp_node)
    nodes  = list(attached_nodes + new_attached_nodes)
    # print("len:",len(nodes),"nodes:",nodes)   \n",                                                       #>>>for logging
    temp_tree = Tree(root= root_node ,nodes = nodes)
    # print( "at last a:" ,attached_nodes ,"u:",unattached_nodes )                                         #>>>for logging
    
    return temp_tree

In [30]:
def create_generation(num_of_population =num_of_population_in_a_gen ):
    trees_list_this_gen = []
    # create the genaration from create_random_tree()
    for i in range(num_of_population):
        temp_tree = create_random_tree()
        trees_list_this_gen.append(temp_tree)
    return trees_list_this_gen

In [58]:
def find_outputs_of_tree(tree:Tree):
    """a function that returns a list of answers of the tree in a list format. (uses for one tree)"""
    answers_list = []
    for x_val in sample_values["x"]:
        try:
            func_str =  str(tree).replace("x" ,str(x_val))
            temp_ans = eval(func_str)
            # temp_ans = tree.evaluate(x_val , node = tree.root)
            answers_list.append(temp_ans)
            # print("for logging:" , tree ,"|||",x_val , answers_list )                                               #>>>for logging
        except ZeroDivisionError:                   #because of division by zero error used this exception.
            answers_list.append(- np.inf)
        except :
            print("ambigues exception was : ", func_str  )
    return answers_list

In [32]:
def find_fitness(functions_answers , real_answers=sample_values["outputs"]):
    """a function that returns fitness of a tree . the real function is saved in a global dictionary named sample_values["outpusts"]"""
    try:
        # fitness = -1* mean_squared_error(real_answers, functions_answers)
        fitness = np.single( np.multiply(-1 ,  np.square(np.subtract(real_answers,functions_answers)).mean()))  #MSE
        # if fitness >10000: fitness = 1000
        # fitness = -1* fitness
        ## if fitness == 0:
        ##     fitness=inf
    except :   
        # print("exception was : " , )                                                                                 
        # raise ValueError("MSE function did not worked correctly.")
        fitness  = -1000000
    return fitness

In [33]:
def crossover(tree1:Tree ,tree2:Tree ):
    """a function that produces 2 children from 2 trees and rarely mutate them."""
    child_1 = copy.deepcopy(tree1)
    child_2 = copy.deepcopy(tree2)
    node_1 = choice(child_1.nodes)
    node_2 = choice(child_2.nodes)
    print(node_1, node_2)
    child_1.replace_node(node_to_replace= node_1  , node_that_will_be=node_2)
    child_2.replace_node(node_to_replace= node_2 , node_that_will_be=node_1)
    if random()>0.95:
        child_1.mutate()
    if random()>0.95:
        child_2.mutate()
    return child_1 , child_2   


In [34]:
def select_to_crossover():
    pass

In [63]:
"""cell that handles the crossovers with functions that defined. """
for gen in range(num_of_gens-1):
    trees_in_gen = create_generation()                                                                               #first generation
    fitness_list = []   #uses as weight in choice()
    # print(trees_in_gen)
    for tree in trees_in_gen:
        trees_func_answers = find_outputs_of_tree(tree)
        trees_fitness = find_fitness(trees_func_answers)
        # print("fitness:{} tree:".format(trees_fitness) , tree)                                          #>>>for logging
        if trees_fitness >=0: print("good tree with fitness:{} found:".format(trees_fitness) , tree)  #>>>for logging . IDK why some fitnesses are big positive integers.
        fitness_list.append(trees_fitness) 
    new_gen = []
    parents_who_survive = choices(trees_in_gen , weights=fitness_list,k=floor(0.2*num_of_population_in_a_gen))       #parent trees that should be in new generation 
    new_gen+=parents_who_survive
    for _ in range(floor(0.8*num_of_population_in_a_gen)):
        tree_1 , tree_2 = choices(trees_in_gen , weights = fitness_list , k=2)



good tree with fitness:55741308.0 found: (((18 * 11) * (45 - 38)) * 47)
good tree with fitness:868113408.0 found: (((48 - x) * 35) * 33)


In [36]:
sample_tree1 = create_random_tree()
sample_tree2 = create_random_tree()
print(sample_tree1 , sample_tree2,sep="~~~~~~~")

(((30 + 25) + 28) * 49)~~~~~~~((39 * x) + ((21 + 46) * (22 / 17)))


In [37]:
f1 = find_fitness(functions_answers=find_outputs_of_tree(sample_tree1))
f2 = find_fitness(functions_answers=find_outputs_of_tree(sample_tree2))
print(f1 ,sample_tree1)
print(f2 ,sample_tree2)

-16276546.0 (((30 + 25) + 28) * 49)
-67661.42 ((39 * x) + ((21 + 46) * (22 / 17)))


In [38]:
child_1 , child_2 =crossover(sample_tree1, sample_tree2)

(((30 + 25) + 28) * 49) 46


In [39]:
print(child_1  , child_2 ,sep="~~~~~~~")

(((30 + 25) + 28) * 49)~~~~~~~((39 * x) + ((21 + (((30 + 25) + 28) * 49)) * (22 / 17)))



# Failed functions and useless codes
----------------------------

In [ ]:
# def calculate(node:Node):
#     # node = tree.root
#     if node.is_operator:
#         return f"( ({calculate(node.left)} ) {node.value} ( {calculate(node.right)}) )"
#     return node.value

In [ ]:
# def create_tree(tree:Tree , parse_input:str ):
#     inputs = parse_input.split(" ")
#     # twos_submultiples = [2**i for i in range(5) ]
#     index = 0
#     tree_spaces = []
#     # make space for each level and fill them with inputs
#     for i in range(5):
#         tmp = []
#         for j in range(2**i) :
#             if index<len(inputs):
#                 tmp.append(inputs[index])
#                 index +=1
#         tree_spaces.append(tmp)
#     index = 0
#     for i in range(5):
#         tmp_index = 0
#         tmp_child_detector = "l"
#         for j in range(i**2):
#             if index<len(inputs):
#                 if tree_spaces[i][j] in operators_set:
#                     if tmp_child_detector =="l":    
#                         tmp_child_detector = "r"
#                     elif tmp_child_detector=="r":
#                         tmp_child_detector = "l"

In [ ]:
# root_node = Node(value="+" )
# first_tree = Tree(root_node)
# create_tree(first_tree, "+ * - 3 x x 4")

In [ ]:
# first_tree.add_left_child("+")
# first_tree.add_left_child("-")

In [ ]:
"""failed create_tree function """
# def create_tree(tree:Tree , parse_input:str , root_node:Node):
#     current_op = None
#     reserved_op = None
#     node = root_node
#     inputs = list(parse_input.split(" "))
#     # current_op =inputs[0]  if inputs[0] in operators_set else None
#     # reserved_op = inputs[1] if inputs[1] in operators_set else None
    
#     num_index = 0
#     while True:
#         if inputs[num_index] not in operators_set :
#             num_index +=1
#         else :
#             break
#     op_index =0
    
#     while(True):
#         if (reserved_op in operators_set):
#             tree.add_left_child(node , inputs[num_index])
#             tree.add_left_child(node , inputs[num_index+1])
#             num_index = num_index+2
#             if inputs[op_index] in operators_set: 
#                 node = 
#                 op_index +=1
#             else:
#                 break


In [ ]:
"""tree_spaces: [[1] [2] [4] [8]]"""

# tree_spaces = []
# for i in range(5):
#     tmp = [None for i in range(2**i)]
#     tree_spaces.append(tmp)
        
# tree_spaces 